In [ ]:
# AutoGluon Text 분석
# https://auto.gluon.ai/stable/tutorials/multimodal/text_prediction/beginner_text.html#sentiment-analysis-task

# Hyperparmeter 최적화
# https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html

In [1]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip3 install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.13.1%2Bcpu-cp38-cp38-linux_x86_64.whl (199.1 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.14.1%2Bcpu-cp38-cp38-linux_x86_64.whl (16.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
  Using cached https://download.pytorch.org/whl/cu116/torch-1.13.1%2Bcu116-cp38-cp38-linux_x86_64.whl (1

In [3]:
!pip3 install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


## 1. 전처리

In [6]:
train["prompt"].isna().sum()

0

In [7]:
# train = train[train["prompt"].isna()==False]

In [8]:
train["answer"].isna().sum()

1

In [9]:
train = train[train["answer"].isna()==False]

In [10]:
train["AI"].isna().sum()

1

In [11]:
train = train[train["AI"].isna()==False]

In [12]:
test.isna().sum()

ID        0
prompt    0
answer    0
dtype: int64

In [13]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


In [14]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import numpy as np

In [16]:
import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
import emoji
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x

In [17]:
train_text = train.copy()
train_text["prompt"]=train["prompt"].apply(lambda x: dataPreprocessing(x))
train_text["answer"]=train["answer"].apply(lambda x: dataPreprocessing(x))

In [18]:
test_text = test.copy()
test_text["prompt"]=test["prompt"].apply(lambda x: dataPreprocessing(x))
test_text["answer"]=test["answer"].apply(lambda x: dataPreprocessing(x))

In [19]:
train_text

,ID,prompt,answer,AI
0,690.0,what is the future of ai,the future of artificial intelligence ai is bo...,1.0
1,304.0,what is your biggest challenge in your career,my biggest challenge in my career is balancing...,0.0
2,63.0,what is the tallest mountain in the world,the tallest mountain in the world is mount eve...,1.0
3,894.0,what are the best super cars,mclaren\nferrari gtb\r\naudi\nmaserati mc\r\n...,0.0
4,796.0,what is deep learning,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,what is the most important thing infamily,the most important thing infamily for me is lo...,0.0
744,895.0,what are the best machine learning models,linear regression\r\nlogistic regression\r\nde...,0.0
745,963.0,how canchop onions without crying,the best thing to do is very simple usevery sh...,0.0
746,156.0,what is the best way to stay healthy,the best way to stay healthy varies from perso...,0.0


In [20]:
from sklearn.model_selection import train_test_split

X = train_text.drop(['ID'], axis=1)
y = train['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
pd.DataFrame(X_train, columns=['prompt', 'answer'])

,prompt,answer
70,what is the name of the unit used to measure p...,the unit used to measure power is the watt
164,what is the name of the largest underwater cav...,sac actun cave system
712,what is the name of the gas that makes up the ...,the gas that makes up the majority of the eart...
265,what is the name of the largest underwater cav...,after doinglot of researchfound that sac actun...
250,what is the most important factor in makingdec...,the most important factor in makingdecision fo...
...,...,...
71,what is your biggest strength,my biggest strength is my ability to communica...
106,what is the currency of australia,australian dollar
270,what is the best way to stay positive,the best way to stay positive in my opinion is...
435,what is the name of the largest underwater cav...,sac actun cave system


In [22]:
pd.DataFrame(y_train, columns=['AI'])

,AI
70,1.0
164,0.0
712,0.0
265,0.0
250,0.0
...,...
71,0.0
106,0.0
270,0.0
435,0.0


In [23]:
train_data = pd.concat([pd.DataFrame(X_train, columns=['prompt', 'answer']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
train_data

,prompt,answer,AI
70,what is the name of the unit used to measure p...,the unit used to measure power is the watt,1.0
164,what is the name of the largest underwater cav...,sac actun cave system,0.0
712,what is the name of the gas that makes up the ...,the gas that makes up the majority of the eart...,0.0
265,what is the name of the largest underwater cav...,after doinglot of researchfound that sac actun...,0.0
250,what is the most important factor in makingdec...,the most important factor in makingdecision fo...,0.0
...,...,...,...
71,what is your biggest strength,my biggest strength is my ability to communica...,0.0
106,what is the currency of australia,australian dollar,0.0
270,what is the best way to stay positive,the best way to stay positive in my opinion is...,0.0
435,what is the name of the largest underwater cav...,sac actun cave system,0.0


In [24]:
test_data = pd.concat([pd.DataFrame(X_test, columns=['prompt', 'answer']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
test_data

,prompt,answer,AI
208,what is the most important lesson you have lea...,the most important lessonhave learned in life ...,0.0
259,what is your biggest challenge in life,my biggest challenge in life is balancing my w...,0.0
97,where to put code documentation,some use other systems takelook at pythons sp...,0.0
148,what is the key to success in the culinary arts,the key to success in the culinary arts is cre...,0.0
395,what is your take on the current state of the ...,the current state of the world is complex and ...,1.0
...,...,...,...
554,what is the best way to stay organized,the best way to stay organized varies from per...,0.0
69,what is the name of the largest living structu...,the largest living structure in the world is t...,1.0
522,what is the meaning of life,the meaning of life is subjective and can vary...,0.0
131,what is natural language processing nlp,nlp is an interdisciplinary subfield of lingu...,0.0


## 2. 모델링

In [25]:
label = 'AI'
eval_metric = 'acc'
time_limit = 3600 * 6

In [26]:
from ray import tune

In [36]:
hyperparameters = {
        "optimization.learning_rate": tune.uniform(0.00005, 0.005),
        "optimization.optim_type": tune.choice(["adamw", "sgd"]),
        "optimization.max_epochs": tune.choice(["10", "20", "30"]),
        "model.timm_image.checkpoint_name": tune.choice(["swin_base_patch4_window7_224", "convnext_base_in22ft1k"])
}
hyperparameter_tune_kwargs = {
    "searcher": "bayes", # random
    "scheduler": "ASHA",
    # "num_trials": 2,
}

In [37]:
# 메모리 이슈 해결
import os
import ray

# Set the memory usage threshold to adjust the kill threshold
# os.environ['RAY_memory_usage_threshold'] = str(1 * 1024 * 1024 * 1024)  # 1GB

# Disable worker killing to prevent OOM errors
# os.environ['RAY_memory_monitor_refresh_ms'] = str(0)  # 0ms

# Initialize Ray with additional resources
# ray.init(memory=10 * 1024 * 1024 * 1024, object_store_memory=10 * 1024 * 1024 * 1024, num_cpus=8)

In [ ]:
from autogluon.multimodal import MultiModalPredictor
# 모듈 못찾는 Attribute Error 발생시 런타임 초기화 후 코드 처음부터 재실행
# EX. AttributeError: module 'PIL.Image' has no attribute 'Resampling'
import uuid

# model_path = PATH+f"model/{uuid.uuid4().hex}-automm_sst_QA_tuning_6hours"
# predictor = MultiModalPredictor(label=label, eval_metric=eval_metric, path=model_path)
predictor = MultiModalPredictor(label=label, eval_metric=eval_metric)
predictor.fit(train_data, presets='best_quality', 
              hyperparameters=hyperparameters,
              hyperparameter_tune_kwargs=hyperparameter_tune_kwargs, 
              time_limit=time_limit)

INFO:lightning_fabric.utilities.seed:Global seed set to 123
No path specified. Models will be saved in: "AutogluonModels/ag-20230301_100350/"


Trial name,should_checkpoint,val_acc
131f5d92,True,0.42500001192092896
154e7000,True,0.24166665971279144
1bb83a27,True,0.7583333253860474
1cb25fe2,True,0.24166665971279144
2bfafacc,True,0.24166665971279144
3e872927,True,0.7583333253860474
400c14ae,True,0.7583333253860474
4322c854,True,0.24166665971279144
436307fb,True,0.24166665971279144
443f67cb,True,0.7583333253860474


2023-03-01 10:13:19,161	ERROR ray_trial_executor.py:595 -- Trial e24b48ec: Error starting runner, aborting!
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 593, in start_trial
    return self._start_trial(trial)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 493, in _start_trial
    runner = self._setup_remote_runner(trial)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 372, in _setup_remote_runner
    raise _AbortTrialExecution(
ray.tune.error._AbortTrialExecution: Trainable runner reuse requires reset_config() to be implemented and return True.
2023-03-01 10:14:46,925	ERROR ray_trial_executor.py:595 -- Trial 08111af3: Error starting runner, aborting!
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 593, in start_trial
    return 

## 3. 성능평가

In [ ]:
# 모델 성능평가
# predictor.leaderboard(silent = True)

In [ ]:
test_score = predictor.evaluate(test_data)
print(test_score)

In [ ]:
# model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_text)

In [ ]:
submission['Category'] = model_pred

In [ ]:
submission = submission.astype({'Category':'int64'})

In [ ]:
submission

In [ ]:
submission.to_csv(PATH+'submit/autoGluon_QA_tuning_6hours_submission_230301.csv', index=False)